## From CSV to Postgres

This scripts reads a CSV files using Pandas, do some data manipulation and exports to PostgresSQL via psycopg2.

The credentials for database connection are stored in another python file.

CSV is from [Kaggle](https://www.kaggle.com/tristan581/17k-apple-app-store-strategy-games)

It contains data of 17.007 strategy games on the Apple App Store. It was collected on the 3rd of August 2019, using the iTunes API and the App Store sitemap.

### Import libraries

In [53]:
import pandas as pd
import psycopg2
import datetime as dt

### Connect to Database

In [54]:
from credentials import user,password,database_name,host

In [63]:
conn = psycopg2.connect(f"host={host} dbname={database_name} user={user} password = {password}")
cur = conn.cursor()

### Read CSV 

In [56]:
df = pd.read_csv('csv/appstore_games.csv')

In [57]:
df.head(2)

,URL,ID,Name,Subtitle,Icon URL,Average User Rating,User Rating Count,Price,In-app Purchases,Description,Developer,Age Rating,Languages,Size,Primary Genre,Genres,Original Release Date,Current Version Release Date
0,https://apps.apple.com/us/app/sudoku/id284921427,284921427,Sudoku,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,4.0,3553.0,2.99,NaN,"Join over 21,000,000 of our fans and download ...",Mighty Mighty Good Games,4+,"DA, NL, EN, FI, FR, DE, IT, JA, KO, NB, PL, PT...",15853568.0,Games,"Games, Strategy, Puzzle",11/07/2008,30/05/2017
1,https://apps.apple.com/us/app/reversi/id284926400,284926400,Reversi,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,3.5,284.0,1.99,NaN,"The classic game of Reversi, also known as Oth...",Kiss The Machine,4+,EN,12328960.0,Games,"Games, Strategy, Board",11/07/2008,17/05/2018


## Data transformation 

In [58]:
# Get number of rows and columns
df.shape

(17007, 18)

In [59]:
# Get data type
df.dtypes

URL                              object
ID                                int64
Name                             object
Subtitle                         object
Icon URL                         object
Average User Rating             float64
User Rating Count               float64
Price                           float64
In-app Purchases                 object
Description                      object
Developer                        object
Age Rating                       object
Languages                        object
Size                            float64
Primary Genre                    object
Genres                           object
Original Release Date            object
Current Version Release Date     object
dtype: object

In [60]:
# Transform to date type
df['Original Release Date'] = df['Original Release Date'].astype('datetime64[ns]')

# Column to lowercase
df['Name'] = str(df['Name']).lower()

# Save some columns
df = df[['ID','Price']]

# Save to new csv
df.to_csv('csv/appstore_games_transformed.csv', index= False)

### Import in database

In [65]:
# Create table in Postgres - define datatypes
cur.execute("""CREATE TABLE test(
id integer ,
price text)""")
conn.commit()

In [31]:
# Read the transformed csv file
dd = pd.read_csv('csv/appstore_games_transformed.csv')

In [66]:
# Insert all data into table in database
with open('csv/appstore_games_transformed.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'test', sep=',')
    conn.commit()

### Final check

In [68]:
# Read inserted data from database
postgreSQL_select_Query = "SELECT * FROM test"
cur.execute(postgreSQL_select_Query)
table = cur.fetchall() 
pd.DataFrame(table, columns =['ID','Price']).head()

,ID,Price
0,284921427,2.99
1,284926400,1.99
2,284946595,0.0
3,285755462,0.0
4,285831220,2.99
